本文内容主要来自：
- https://www.jiqizhixin.com/articles/2017-10-28-4
- https://arxiv.org/abs/1710.09829

Capsule是一组神经元，他所对应的活动向量(activity vector)用于代表某个特定类型的实体实例化时所用的参数。我们使用这个activity vector的长度来表示这个实体存在的概率，用这个向量的方向来表示实例化参数。在同一个层的活动的capsules通过转移矩阵，对更高一层的capsule的实例化参数进行预测。在高一层的capsules中，获得多数支持的capsule会被激活。（一层只有一个被激活吗？如此的话应该只会产生一票啊？）。

### 介绍
人类的视觉系统并不会对所看到的事物进行均等的处理。焦点（fixation point固定点）的存在使得我们可以忽略一些不相关的细节，而只对我们关注的小部分的场景进行高精度的处理。（但是其他部分的内容也不应该被完全抛弃啊？？？）。我们很难通过introspection(内省，什么鬼？？)了解到，我们通过场景的一个或多个焦点总共可以获得多少信息，总之，我们在这里默认为这个信息量远大于目标的种类和属性就是了。我们假设，我们的多层视觉系统将为每个单独的焦点建立一个类似于解析树的东西。在这里，我们会忽略基于不同焦点的解析树之间是如何进行交流的。<br>
我们假设，每个解析树由一个多层神经网络来表示。在这个神经网络之中，每一层的神经元节点会被划分成多个小组，每个小组对应一个capusle。不存在不属于任何capsule的神经节点。通过iterative routing的方法，每个capsule会在其下一层选择一个capsule作为其在这个树中的父节点。对于最后一层的capsule来说，这个iterative过程的目的，将是用于解决从部分到整体的问题。(注意每个焦点对应有一棵解析树，而一个场景，一般会存在多个焦点)<br>
一个激活的capsule中，不同神经节点之间的活动(???确定不是变量什么的？？)可以用来表示场景中特定实体的各种属性，包括位置，色彩等等。其中一个比较特殊的属性是指示这个实例化的对象是否存在。关于这个属性，一种简单的表示方法是使用一个逻辑神经元。但是在这里，我们使用另外一种方法，我们使用活动向量的长度，来表示这个属性。为了使得这个向量的长度不会超过1，我们使用了非线性的方法，使得在保持向量的长度不变的前提下，对其长度进行缩放。<br>
capsule的输出使一个向量的这个事实，确保了我们可以通过动态routing的方法，把这个capsule的输出送到适合的父节点中。刚开始，所有的下一层的capsule都会接受到这个输出（即所有人都是老爸）。但通过耦合系数（其和为1），我们将缩小可能的父节点的数量。这对每个入围的父类节点，capsule会为其提供一个prediction vector。这个向量可以通过将capsule的输出和权重矩阵相乘得到。加入某个父类节点的输出和这个prediction vector的标量积很大，那么我们将提高capsule和这个父类节点之间的耦合系数，降低capsule和其他父类节点之间的耦合系数。这将会进一步加强capsule对这个父类节点的影响，从而提高最后的标量积。这种机制会比pooling机制更加有效率，因为在pooling机制中，将忽略出选中父节点外的所有其他的节点。<br>
CNN机制实现了特征检测器在不同位置的重复使用，而这种机制被证明了对于图片分析很有帮助。因此，我们也尝试结合这种机制。我们使用会输出向量的capsule来替代CNN中输出标量的特征检测器。使用前面提到的routing机制来替代pooling。为了使得这个解析树可以被应用到图片中不同的位置，我们使得，除了最后一层的capsule外的其他capsule都为卷积元。<br>
和 CNN 一样，我们创建更高级别的 capsule 来覆盖图像的更大区域，但是与最大池化不同，我们无法抛弃区域内实体的确切位置信息。对于低级 capsule 而言，位置信息由活跃的 capsule 进行「位置编码」（place-coded）。随着层级的上升，越来越多的位置信息层级由 capsule 的输出向量中的实数值部分进行「率编码」（rate-coded）。从位置编码到率编码的转变和高级别 capsule 代表具备更多自由度的更复杂实体共同表明 capsule 的维度应该随着层级的上升而增长。